# RAG Gamit ang Azure AI Agent Service at Semantic Kernel

Ang code snippet na ito ay nagpapakita kung paano lumikha at mag-manage ng isang Azure AI agent para sa retrieval-augmented generation (RAG) gamit ang `Azure AI Agent Service` at `Semantic Kernel`. Pinoproseso ng agent ang mga tanong ng user batay sa narekober na konteksto at nagbibigay ng tamang sagot ayon dito.


SQLite Bersyon na Ayusin  
Kung makaranas ka ng error:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

Tanggalin ang pagkaka-comment ng code block na ito sa simula ng iyong notebook:  


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### Pag-import ng mga Package
Ang sumusunod na code ay nag-i-import ng mga kinakailangang package:


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# Retrieval-Augmented Generation gamit ang Semantic Kernel at Azure AI Agent Service

Ipinapakita ng halimbawang ito kung paano gamitin ang **Azure AI Agent Service** upang magsagawa ng **Retrieval-Augmented Generation (RAG)** sa pamamagitan ng pagsasama ng isang language model sa domain-specific na konteksto mula sa isang na-upload na dokumento.

### Paano Ito Gumagana

1. **Pag-upload ng Dokumento**: Isang markdown file (document.md) na naglalaman ng impormasyon (patakaran sa travel insurance ng Contoso) ang ina-upload sa agent service.

2. **Paglikha ng Vector Store**: Ang dokumento ay ini-index sa isang vector store upang paganahin ang semantic search sa nilalaman nito.

3. **Pag-configure ng Agent**: Ang isang agent ay ini-instantiate gamit ang modelong `gpt-4o` na may sumusunod na mahigpit na mga tagubilin:
   - Sumagot lamang ng mga tanong batay sa narekober na nilalaman mula sa dokumento.
   - Tumangging sumagot kung ang tanong ay wala sa saklaw.

4. **Pagsasama ng File Search Tool**: Ang `FileSearchTool` ay nirehistro sa agent, na nagbibigay-daan sa modelong maghanap at kumuha ng mga kaugnay na bahagi mula sa na-index na dokumento habang nasa proseso ng inference.

5. **Interaksyon ng Gumagamit**: Maaaring magtanong ang mga gumagamit. Kung may makukuhang kaugnay na impormasyon mula sa dokumento, ang agent ay bumubuo ng sagot na may basehan.  
   Kung wala, malinaw na sasabihin ng agent na walang sapat na impormasyon sa dokumento.


### Pangunahing Function

Ang pangunahing function ay ang entry point ng iyong programa. Dito nagsisimula ang lahat ng lohika at operasyon.

```python
def main():
    # Simulan ang pangunahing lohika dito
    print("Maligayang pagdating sa programa!")
```

[!NOTE] Siguraduhing tawagin ang `main()` function upang masimulan ang programa.

### Paano Tawagin ang Main Function

Upang masiguro na ang `main()` function ay tatakbo lamang kapag ang script ay direktang pinatakbo, gamitin ang sumusunod na pattern:

```python
if __name__ == "__main__":
    main()
```

[!IMPORTANT] Ang pattern na ito ay mahalaga upang maiwasan ang hindi sinasadyang pagpapatakbo ng code kapag ang script ay na-import bilang isang module.

### Mga Halimbawa ng Output

Kapag pinatakbo mo ang script, makikita mo ang sumusunod na output sa terminal:

```
Maligayang pagdating sa programa!
```

[!TIP] Maaari mong i-customize ang mensahe sa loob ng `print()` function upang umangkop sa iyong mga pangangailangan.


Siguraduhing patakbuhin muna ang `az login` gamit ang Azure CLI upang maibigay ang tamang authentication context habang ginagamit ang `DefaultAzureCredential`. Ang Azure AI Agent Service ay hindi gumagamit ng mga API key.


In [ ]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        file: OpenAIFile = await client.agents.upload_file_and_poll(file_path="document.md", purpose="assistants")
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

Naiintindihan ko. Mangyaring ibigay ang Markdown file na nais mong isalin sa Tagalog.



---

**Paunawa**:  
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagama't sinisikap naming maging tumpak, pakitandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o hindi pagkakatugma. Ang orihinal na dokumento sa orihinal nitong wika ang dapat ituring na opisyal na sanggunian. Para sa mahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na maaaring magmula sa paggamit ng pagsasaling ito.
